In [2]:
# pip install hana-ml
from hana_ml import dataframe

ModuleNotFoundError: No module named 'shapely'


In [2]:
from hana_ml import dataframe

ModuleNotFoundError: No module named 'shapely'


In [3]:
conn = dataframe.ConnectionContext(address="a129e995-340b-43db-a7f4-d1949f62c7c3.hana.prod-eu10.hanacloud.ondemand.com",
                                       port=443,
                                       user="KARLSCHNEIDER",
                                       password="Welcome04")

In [4]:
# df = conn.table('MY_TABLE', schema='KARLSCHNEIDER')
df = dataframe.DataFrame(conn, 'select CURRENT_TIMESTAMP,CURRENT_DATE,CURRENT_USER,CURRENT_SCHEMA from DUMMY')
pandas_df = df.collect()
print(pandas_df)

        CURRENT_TIMESTAMP CURRENT_DATE   CURRENT_USER CURRENT_SCHEMA
0 2023-11-27 08:52:10.451   2023-11-27  KARLSCHNEIDER  KARLSCHNEIDER


In [9]:
# INPUTS
#REMOTE_SOURCE_NAME='I531155_ODataAdapter_S4_INTEGRATION'
#FILTER_CLAUSE='A_ProductSalesT%'
REMOTE_DATABASE_NAME='<NULL>'
REMOTE_SCHEMA_NAME='<NULL>'
REMOTE_SOURCE_NAME='I531155_ABAPAdapter_S4_INTEGRATION'
FILTER_CLAUSE='ABAPTABLES.MARA_'
#FILTER_CLAUSE='ABAPTABLES.MAR%'

In [ ]:
# Create Dictionary on Remote Source to Fill REMOTE_SOURCE_OBJECTS System View
#conn.execute_sql('ALTER REMOTE SOURCE "' + REMOTE_SOURCE_NAME + '" REFRESH OBJECTS')
# Clear Dictionary
#conn.execute_sql('ALTER REMOTE SOURCE "' REMOTE_SOURCE_NAME + '"" CLEAR OBJECTS')

In [10]:
# Get All Remote Objects that are Starting with A_ProductP in Remote Source I531155_ODataAdapter_S4_INTEGRATION  
#df = dataframe.DataFrame(conn, 'SELECT REMOTE_SOURCE_NAME, OBJECT_NAME FROM REMOTE_SOURCE_OBJECTS where REMOTE_SOURCE_NAME = \'I531155_ODataAdapter_S4_INTEGRATION\' and OBJECT_NAME like \'A_ProductSalesT%\'')
df = dataframe.DataFrame(conn, 'SELECT REMOTE_SOURCE_NAME, OBJECT_NAME FROM REMOTE_SOURCE_OBJECTS where REMOTE_SOURCE_NAME =\'' + REMOTE_SOURCE_NAME +'\'and OBJECT_NAME like \''+FILTER_CLAUSE +'\'')


# Convert HANA Dataframe to pandas Dataframe
pandas_df = df.collect()
print(pandas_df)
print('')

                    REMOTE_SOURCE_NAME                OBJECT_NAME
0   I531155_ABAPAdapter_S4_INTEGRATION  ABAPTABLES.MARA_DRAD_STXL
1   I531155_ABAPAdapter_S4_INTEGRATION       ABAPTABLES.MARA_STXL
2   I531155_ABAPAdapter_S4_INTEGRATION       ABAPTABLES.MARC_STXL
3   I531155_ABAPAdapter_S4_INTEGRATION       ABAPTABLES.MARA_AUSP
4   I531155_ABAPAdapter_S4_INTEGRATION           ABAPTABLES.MARCH
..                                 ...                        ...
83  I531155_ABAPAdapter_S4_INTEGRATION           ABAPTABLES.MARA1
84  I531155_ABAPAdapter_S4_INTEGRATION       ABAPTABLES.MARA_KSSK
85  I531155_ABAPAdapter_S4_INTEGRATION           ABAPTABLES.MARCU
86  I531155_ABAPAdapter_S4_INTEGRATION            ABAPTABLES.MARV
87  I531155_ABAPAdapter_S4_INTEGRATION            ABAPTABLES.MARU

[88 rows x 2 columns]



In [11]:
for index, row in pandas_df.iterrows():
    # Test Interation 
    print('Object: '+ row["OBJECT_NAME"])
    print('')

    # Create Virtual Table for all Objects in Data Frame (all Remote Objects with previously applied filter)
    #conn.create_virtual_table(table='VT_'+row["OBJECT_NAME"], remote_source='I531155_ODataAdapter_S4_INTEGRATION', remote_database='<NULL>', remote_schema='<NULL>', remote_table=row["OBJECT_NAME"])
    conn.create_virtual_table(table='VT_'+row["OBJECT_NAME"], remote_source=REMOTE_SOURCE_NAME, remote_database=REMOTE_DATABASE_NAME, remote_schema=REMOTE_SCHEMA_NAME, remote_table=row["OBJECT_NAME"])

    # Print Top 1 Row for every created virtual table for confirmation
    df1 = dataframe.DataFrame(conn,'SELECT TOP 1 * FROM "VT_'+row["OBJECT_NAME"]+'"')
    pd1 = df1.collect()
    print(pd1)
    print('')

    # Drop every created Virtual Table
    conn.drop_table(table='VT_'+row["OBJECT_NAME"])

Object: ABAPTABLES.MARA_DRAD_STXL

Empty DataFrame
Columns: [MANDT, MATNR, TDOBJECT, TDNAME, TDID, TDSPRAS, LINE_COUNTER, DOKAR, DOKNR, DOKVR, DOKTL, DOKOB, OBZAE, OBJKY, TDFORMAT, TDLINE]
Index: []

Object: ABAPTABLES.MARA_STXL

Empty DataFrame
Columns: [MANDT, MATNR, TDOBJECT, TDNAME, TDID, TDSPRAS, LINE_COUNTER, TDFORMAT, TDLINE]
Index: []

Object: ABAPTABLES.MARC_STXL

Empty DataFrame
Columns: [MANDT, MATNR, WERKS, TDOBJECT, TDNAME, TDID, TDSPRAS, LINE_COUNTER, TDFORMAT, TDLINE]
Index: []

Object: ABAPTABLES.MARA_AUSP

Empty DataFrame
Columns: [MANDT, MATNR, ATINN, ATZHL, MAFID, KLART, ADZHL, ATWRT, ATFLV, ATAWE, ATFLB, ATAW1, ATCOD, ATTLV, ATTLB, ATPRZ, ATINC, ATAUT, AENNR, DATUV, LKENZ, ATIMB, ATZIS, ATSRT, ATVGLART, DATUB, DEC_VALUE_FROM, DEC_VALUE_TO, CURR_VALUE_FROM, CURR_VALUE_TO, CURRENCY, DATE_FROM, DATE_TO, TIME_FROM, TIME_TO]
Index: []

[0 rows x 35 columns]

Object: ABAPTABLES.MARCH

Empty DataFrame
Columns: [MANDT, MATNR, WERKS, LFGJA, LFMON, UMLMC, TRAME, VKUMC, VKTRW,

(403, 'internal error: Error opening the cursor for the remote database <I531155_ABAPAdapter_S4_INTEGRATION> TABLE_WITHOUT_DATA\n')
Failed to retrieve data for the current dataframe, (403, 'internal error: Error opening the cursor for the remote database <I531155_ABAPAdapter_S4_INTEGRATION> TABLE_WITHOUT_DATA\n')


Error: (403, 'internal error: Error opening the cursor for the remote database <I531155_ABAPAdapter_S4_INTEGRATION> TABLE_WITHOUT_DATA\n')

In [6]:
# Close Connection to SAP HANA
conn.close()